<a href="https://colab.research.google.com/github/Small-Fiend/DDSA/blob/main/DS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from dataclasses import dataclass
from typing import Tuple, List
import numpy as np

In [27]:
def gen_node_positions(m):
  return np.unique(np.random.randit(low=0, hight=2**m, size=m)).tolist()

In [28]:
# Количество битов для идентификатора
M = 3
node_pos: list = [0, 1, 3]
print(node_pos)

[0, 1, 3]


In [29]:
CHORD_CIRCLE = []


@dataclass
class Finger:
    start: int
    interval: Tuple[int, int]
    node_id: int

        
def within_interval(id, a, b, a_included=False, b_included=False):
    """
    Проверка, может ли finger принимать запросы относительно этого идентификатора
    """
    print(f'Check {id} in [{a}, {b}]')
    is_less_than_b = id <= b if b_included else id < b
    is_less_than_a = id <= a if a_included else id < a
    is_more_than_a = id >= a if a_included else id > a
    if a < b:
        return is_more_than_a and is_less_than_b
    elif a > b:
        return is_more_than_a or is_less_than_b
    else:
        return a_included or b_included


call_depth = 0


def log(fn):
    def decorated_fn(self, *args, **kwargs):
        global call_depth
        tab = lambda msg: '--' * call_depth + msg
        print(tab('-' * 10))
        print(tab(f'Start call {fn.__name__}.'))
        print(tab(f'Information: node.id = {self.id}, args = {args}, kwargs = {kwargs}'))
        call_depth += 1
        result = fn(self, *args, **kwargs)
        call_depth -= 1
        print(tab(f'Finish call {fn.__name__}.'))
        print(tab(f'Returned value: {result}'))
        print(tab('/' * 10))
        return result
    return decorated_fn


class ChordNode:
    def __init__(self, id, finger_table, chord_service, predecessor_id):
        self.id = id
        self.finger_table = finger_table
        self.chord_service = chord_service
        self.chord_service.add_node(self)
        self.predecessor_id = predecessor_id
    
    @property
    @log
    def successor(self):
        successor_id = self.finger_table[0].node_id
        return self.chord_service.get_node(successor_id)
    
    @property
    @log
    def predecessor(self):
        return self.chord_service.get_node(self.predecessor_id)
    
    @property
    @log
    def successor_id(self):
        successor_id = self.finger_table[0].node_id
        return successor_id
    
    # Поиск
    
    @log
    def find_successor(self, id):
        if self.id == id:
            return self
        
        node = self.find_predecessor(id)
        return node.successor
    
    @log
    def find_predecessor(self, id):
        if self.id == id:
            return self.predecessor
        
        node = self
        while not within_interval(id, node.id, node.successor_id, b_included=True):
            node = node.closest_preceding_finger(id)
        return node
    
    @log
    def closest_preceding_finger(self, id):
        for i in range(len(self.finger_table) - 1, -1, -1):
            finger_id = self.finger_table[i].node_id
            print(f'Checking finger {i}')
            if within_interval(finger_id, self.id, id):
                node = self.chord_service.get_node(finger_id)
                if node is None:
                    self.finger_table[i].node_id = self.id
                    continue
                return self.chord_service.get_node(finger_id)
        return self
    
    # Присоединение
    
    @log
    def join(self, other_node):
         # Создание пустой таблицы
        self.prepare_finger_table()
        if other_node is not None:
            # Использование `other_node` для поиска fingers
            self.init_finger_table(other_node)
            self.update_others()
        else:
            # Это единтсвенный узел в сети
            for i in range(self.chord_service.capacity):
                self.finger_table[i].node_id = self.id
            self.predecessor_id = self.id
                
    @log
    def prepare_finger_table(self):
        self.finger_table = []
        m = self.chord_service.capacity
        for i in range(m):
            finger = Finger(None, None, None)
            finger.start = (self.id + 2**i) % 2 ** m
            interval_end = (self.id + 2**(i+1)) % 2 ** m
            finger.interval = (finger.start, interval_end)
            self.finger_table.append(finger)
    
    @log
    def init_finger_table(self, other_node):
        first_finger_start = self.finger_table[0].start
        successor = other_node.find_successor(first_finger_start)
        self.finger_table[0].node_id = successor.id
        self.predecessor_id = successor.predecessor_id
        successor.predecessor_id = self.id
        
        for i in range(self.chord_service.capacity - 1):
            pre_finger = self.finger_table[i]
            finger = self.finger_table[i + 1]
            if within_interval(finger.start, self.id, pre_finger.node_id, a_included=True):
                finger.node_id = pre_finger.node_id
            else:
                finger.node_id = other_node.find_successor(finger.start).id

    @log            
    def update_others(self, removed_id=None):
        m = self.chord_service.capacity
        for i in range(m):
            pred_id = (self.id - 2**i + 1) % 2**m
            p = self.find_predecessor(pred_id)
            if removed_id is not None and p.id == removed_id:
                p = self.find_predecessor(removed_id)

            p.update_finger_table_v2(self.id, i)

    @log
    def update_finger_table(self, s, i): 
        if s == self.id:
            return
        
        finger = self.finger_table[i]
        if within_interval(s, self.id, finger.node_id, a_included=True):
            finger.node_id = s
            print(f'Updated finger {i} at {self.id} with {s}')
            p = self.predecessor_id
            p_node = self.chord_service.get_node(p)
            p_node.update_finger_table(s, i)
            
    @log
    def disconnect(self):
        self.chord_service.remove_node(self.id)
        # Обновление ближайших соседей
        succ = self.successor
        pred = self.predecessor
        succ.predecessor_id = pred.id
        pred.finger_table[0].node_id = succ.id
        
        self.successor.update_others(self.id)

    @log
    def update_finger_table_v2(self, s, i): 
        if s == self.id:
            return
        
        finger = self.finger_table[i]
        not_accessible = self.chord_service.get_node(finger.node_id) is None
        print(f'Updating finger {i} with {s}. not_accessible={not_accessible}')
        if within_interval(s, self.id, finger.node_id, a_included=True) or not_accessible:
            finger.node_id = s
            print(f'Updated finger {i} at {self.id} with {s}')
            p = self.predecessor_id
            p_node = self.chord_service.get_node(p)
            p_node.update_finger_table_v2(s, i)
        
        if finger.node_id == s:
            print('Found old finger which is already correct. Propagating info further...')
            p = self.predecessor_id
            p_node = self.chord_service.get_node(p)
            p_node.update_finger_table_v2(s, i)
        
    def __repr__(self):
        return f'Node: id={self.id}'

In [30]:
class ChordService:
  def __init__(self, m: int):
    """
    Инициализация сервиса chord. Если указано `chord_circle`, 
    он используется для инициализации сервиса, в противном случае узлы будут
    генерироваться случайным образом.

    Параметры: 
    m: int
      m = log2(максимальное число узлов)
    """
    self.n_nodes_max = m
    self.chord_circle = {}

  def add_node(self, node: ChordNode):
    """
    Добавление узла
    
    Параметры:
    key: int
        Ключевой идентификатор
    """
    self.chord_circle[node.id] = node

  def get_node(self, id) -> ChordNode:
    return self.chord_circle.get(id)

  def remove_node(self, id):
    self.chord_circle.pop(id)

  def init_chord_circle(self, circle):
    for node in circle:
      self.chord_circle[node.id] = node

  @property
  def capacity(self):
    return self.n_nodes_max

In [31]:
service = ChordService(3)

In [32]:
node_0 = ChordNode(
    id = 0,
    finger_table = None,
    chord_service = service,
    predecessor_id = None
)

In [33]:
node_0.join(None)

----------
Start call join.
Info: node.id=0, args=(None,), kwargs={}
------------
--Start call prepare_finger_table.
--Info: node.id=0, args=(), kwargs={}
--Finish call prepare_finger_table.
--Returned value: None
--//////////
Finish call join.
Returned value: None
//////////


In [34]:
node_1 = ChordNode(
    id = 1, 
    finger_table = None,
    chord_service = service,
    predecessor_id = None
)

In [35]:
node_1.join(node_0)

----------
Start call join.
Info: node.id=1, args=(Node: id=0,), kwargs={}
------------
--Start call prepare_finger_table.
--Info: node.id=1, args=(), kwargs={}
--Finish call prepare_finger_table.
--Returned value: None
--//////////
------------
--Start call init_finger_table.
--Info: node.id=1, args=(Node: id=0,), kwargs={}
--------------
----Start call find_successor.
----Info: node.id=0, args=(2,), kwargs={}
----------------
------Start call find_predecessor.
------Info: node.id=0, args=(2,), kwargs={}
------------------
--------Start call successor_id.
--------Info: node.id=0, args=(), kwargs={}
--------Finish call successor_id.
--------Returned value: 0
--------//////////
Check 2 in [0, 0]
------Finish call find_predecessor.
------Returned value: Node: id=0
------//////////
----------------
------Start call successor.
------Info: node.id=0, args=(), kwargs={}
------Finish call successor.
------Returned value: Node: id=0
------//////////
----Finish call find_successor.
----Returned

In [36]:
node_1.finger_table

[Finger(start=2, interval=(2, 3), node_id=0),
 Finger(start=3, interval=(3, 5), node_id=0),
 Finger(start=5, interval=(5, 1), node_id=0)]

In [37]:
node_0.finger_table

[Finger(start=1, interval=(1, 2), node_id=1),
 Finger(start=2, interval=(2, 4), node_id=0),
 Finger(start=4, interval=(4, 0), node_id=0)]

In [38]:
node_3 = ChordNode(
    id = 3,
    finger_table = None,
    chord_service = service,
    predecessor_id = None
)

In [39]:
node_3.join(node_0)

----------
Start call join.
Info: node.id=3, args=(Node: id=0,), kwargs={}
------------
--Start call prepare_finger_table.
--Info: node.id=3, args=(), kwargs={}
--Finish call prepare_finger_table.
--Returned value: None
--//////////
------------
--Start call init_finger_table.
--Info: node.id=3, args=(Node: id=0,), kwargs={}
--------------
----Start call find_successor.
----Info: node.id=0, args=(4,), kwargs={}
----------------
------Start call find_predecessor.
------Info: node.id=0, args=(4,), kwargs={}
------------------
--------Start call successor_id.
--------Info: node.id=0, args=(), kwargs={}
--------Finish call successor_id.
--------Returned value: 1
--------//////////
Check 4 in [0, 1]
------------------
--------Start call closest_preceding_finger.
--------Info: node.id=0, args=(4,), kwargs={}
Checking finger 2
Check 0 in [0, 4]
Checking finger 1
Check 0 in [0, 4]
Checking finger 0
Check 1 in [0, 4]
--------Finish call closest_preceding_finger.
--------Returned value: Node: id

In [40]:
node_3.finger_table

[Finger(start=4, interval=(4, 5), node_id=0),
 Finger(start=5, interval=(5, 7), node_id=0),
 Finger(start=7, interval=(7, 3), node_id=0)]

In [41]:
node_0 = ChordNode(
    id = 0,
    finger_table = [
      Finger(1, [1, 2], 1),
      Finger(2, [2, 4], 3),
      Finger(4, [4, 0], 0)
    ],
    chord_service = service,
    predecessor_id = 3
)

In [42]:
node_1 = ChordNode(
    id = 1,
    finger_table = [
        Finger(2, [2, 3], 3), 
        Finger(3, [3, 5], 3),
        Finger(5, [5, 1], 0)
    ],
    chord_service = service,
    predecessor_id = 0
)

In [43]:
node_3 = ChordNode(
    id = 3,
    finger_table = [
        Finger(4, [4, 5], 0), 
        Finger(5, [5, 7], 0),
        Finger(7, [7, 3], 0)
    ],
    chord_service = service,
    predecessor_id = 1
)

In [44]:
node_3.find_successor(2).id

----------
Start call find_successor.
Info: node.id=3, args=(2,), kwargs={}
------------
--Start call find_predecessor.
--Info: node.id=3, args=(2,), kwargs={}
--------------
----Start call successor_id.
----Info: node.id=3, args=(), kwargs={}
----Finish call successor_id.
----Returned value: 0
----//////////
Check 2 in [3, 0]
--------------
----Start call closest_preceding_finger.
----Info: node.id=3, args=(2,), kwargs={}
Checking finger 2
Check 0 in [3, 2]
----Finish call closest_preceding_finger.
----Returned value: Node: id=0
----//////////
--------------
----Start call successor_id.
----Info: node.id=0, args=(), kwargs={}
----Finish call successor_id.
----Returned value: 1
----//////////
Check 2 in [0, 1]
--------------
----Start call closest_preceding_finger.
----Info: node.id=0, args=(2,), kwargs={}
Checking finger 2
Check 0 in [0, 2]
Checking finger 1
Check 3 in [0, 2]
Checking finger 0
Check 1 in [0, 2]
----Finish call closest_preceding_finger.
----Returned value: Node: id=1
-

3

In [45]:
node_4 = ChordNode(
    id = 4,
    finger_table = None,
    chord_service = service,
    predecessor_id = None
)

In [46]:
node_4.join(node_3)

----------
Start call join.
Info: node.id=4, args=(Node: id=3,), kwargs={}
------------
--Start call prepare_finger_table.
--Info: node.id=4, args=(), kwargs={}
--Finish call prepare_finger_table.
--Returned value: None
--//////////
------------
--Start call init_finger_table.
--Info: node.id=4, args=(Node: id=3,), kwargs={}
--------------
----Start call find_successor.
----Info: node.id=3, args=(5,), kwargs={}
----------------
------Start call find_predecessor.
------Info: node.id=3, args=(5,), kwargs={}
------------------
--------Start call successor_id.
--------Info: node.id=3, args=(), kwargs={}
--------Finish call successor_id.
--------Returned value: 0
--------//////////
Check 5 in [3, 0]
------Finish call find_predecessor.
------Returned value: Node: id=3
------//////////
----------------
------Start call successor.
------Info: node.id=3, args=(), kwargs={}
------Finish call successor.
------Returned value: Node: id=0
------//////////
----Finish call find_successor.
----Returned

In [47]:
node_4.finger_table

[Finger(start=5, interval=(5, 6), node_id=0),
 Finger(start=6, interval=(6, 0), node_id=0),
 Finger(start=0, interval=(0, 4), node_id=0)]

In [48]:
node_3.finger_table

[Finger(start=4, interval=[4, 5], node_id=4),
 Finger(start=5, interval=[5, 7], node_id=0),
 Finger(start=7, interval=[7, 3], node_id=0)]

In [49]:
node_1.finger_table

[Finger(start=2, interval=[2, 3], node_id=3),
 Finger(start=3, interval=[3, 5], node_id=3),
 Finger(start=5, interval=[5, 1], node_id=0)]

In [50]:
node_0.finger_table

[Finger(start=1, interval=[1, 2], node_id=1),
 Finger(start=2, interval=[2, 4], node_id=3),
 Finger(start=4, interval=[4, 0], node_id=4)]

In [51]:
node_4.disconnect

<bound method log.<locals>.decorated_fn of Node: id=4>